In [1]:
import os
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from scipy.sparse import csr_matrix
import sys
import re

### Write a python script to convert the data into a dataframe (check the section below for the head and tail of dataframe)

In [2]:
# Specify the folder path containing the text files
folder_path = r"C:\Users\mitra\Desktop\INNOMATICS(MITRABHANU PANDA)\GEN AI (ADVANCE)\5. ASSIGNMENTS\ASSIGNMENT-1\text"

# Initialize an empty list to store the data
data = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        
        # Open and read the file
        with open(file_path, 'r') as file:
            content = file.read().strip()
            # Append the filename and content to the list
            data.append([filename, content])

# Convert the list to a DataFrame
df = pd.DataFrame(data, columns=['Filename', 'Content'])


# Split & create a Label Column
df["label"]=df["Content"].apply(lambda x:x.split("\n")[0])

# Split & create a text Column
df["text"]=df["Content"].apply(lambda x:x.split("\n")[1])

# Drop the unwanted Column
df.drop(["Content","Filename"],axis=1,inplace=True)

# Show the Dataframe
df


,label,text
0,MajorClaim,we should attach more importance to cooperatio...
1,Premise,Take Olympic games which is a form of competit...
2,Premise,The high technology and new ideas applied into...
3,Premise,pollutions are not just caused by the burning ...
4,Premise,the improvements of work efficiency also attri...
...,...,...
6084,MajorClaim,addressing pollution and traffic issues only b...
6085,Premise,whether it can work out for alleviating traffi...
6086,Premise,price control institution has been used in ple...
6087,Claim,it seems not easy to increase petrol price ins...


In [3]:
df.head() 

,label,text
0,MajorClaim,we should attach more importance to cooperatio...
1,Premise,Take Olympic games which is a form of competit...
2,Premise,The high technology and new ideas applied into...
3,Premise,pollutions are not just caused by the burning ...
4,Premise,the improvements of work efficiency also attri...


In [4]:
df.tail()

,label,text
6084,MajorClaim,addressing pollution and traffic issues only b...
6085,Premise,whether it can work out for alleviating traffi...
6086,Premise,price control institution has been used in ple...
6087,Claim,it seems not easy to increase petrol price ins...
6088,Premise,governments have a macro-economic perspective ...


### In what format are the observations available in the given raw data?
   * Ans :--> text files

### In what format are the observations available in the given raw data?

   * Ans :--> 6089

### How many files are having 'Premise' label?

In [5]:
df["label"].value_counts()["Premise"]

3832

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mitra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### What is the maximum number of character level tokens in a document in raw 'text' column?

In [7]:
# Function to tokenize text at the character level
def char_tokenize(text):
    return list(text)

# Apply the tokenizer to each document and calculate the length of the token list
df['char_token_count'] = df['text'].apply(lambda x: len(char_tokenize(x)))

# Find the maximum number of character-level tokens in the 'text' column
max_char_tokens = df['char_token_count'].max()

print(f"The maximum number of character-level tokens in a document is: {max_char_tokens}")

The maximum number of character-level tokens in a document is: 344


### What is the maximum number of word level tokens in a document in raw 'text' columns?

In [8]:
df["word_token_count"] = df["text"].apply(lambda x: len(x.split()))
max_words_token = df['word_token_count'].max()
print("Max Number of Word Level Tokens: ",max_words_token)

Max Number of Word Level Tokens:  67


In [9]:
df

,label,text,char_token_count,word_token_count
0,MajorClaim,we should attach more importance to cooperatio...,72,10
1,Premise,Take Olympic games which is a form of competit...,297,58
2,Premise,The high technology and new ideas applied into...,154,26
3,Premise,pollutions are not just caused by the burning ...,137,20
4,Premise,the improvements of work efficiency also attri...,127,19
...,...,...,...,...
6084,MajorClaim,addressing pollution and traffic issues only b...,84,13
6085,Premise,whether it can work out for alleviating traffi...,92,15
6086,Premise,price control institution has been used in ple...,131,23
6087,Claim,it seems not easy to increase petrol price ins...,52,9


### Applying Text Cleaning

#### Cleaning Steps:
 - A valid token should only contain alphanumeric and "."
 - Convert to lower
 - word tokenize
 - stop word removal
 - lemmatization

In [10]:
def clean(doc): 
    # doc is a string of text
    
    # Let's define a regex to match special characters and digits
    regex = "[^a-zA-Z.]"
    doc = re.sub(regex, " ", doc)

    # Convert to lowercase
    doc = doc.lower()
        
    # Tokenization
    tokens = nltk.word_tokenize(doc)

    # Stop word removal
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    
    # Join and return
    return " ".join(lemmatized_tokens)

In [11]:
df['clean_text'] = df['text'].apply(lambda x : clean(x))

df.head()

,label,text,char_token_count,word_token_count,clean_text
0,MajorClaim,we should attach more importance to cooperatio...,72,10,attach importance cooperation primary education
1,Premise,Take Olympic games which is a form of competit...,297,58,take olympic game form competition instance ha...
2,Premise,The high technology and new ideas applied into...,154,26,high technology new idea applied practice may ...
3,Premise,pollutions are not just caused by the burning ...,137,20,pollution caused burning oil chemical pollutan...
4,Premise,the improvements of work efficiency also attri...,127,19,improvement work efficiency also attribute spe...


### After applying text cleaning, what is the maximum number of word level tokens in 'clean_text' column?

In [12]:
df['word_token_count_clean'] = df['clean_text'].apply(lambda x: len(word_tokenize(x)))

max_word_tokens_clean = df['word_token_count_clean'].max()

print(f"The maximum number of word-level tokens in a document is: {max_word_tokens_clean}")

The maximum number of word-level tokens in a document is: 31


In [13]:
df["clean_text"][0]

'attach importance cooperation primary education'

In [14]:
df['clean_text'].apply(lambda x: len(word_tokenize(x)))[0]

5

In [15]:
def clean1(doc): 
    # doc is a string of text
    
    # Let's define a regex to match special characters and digits
    regex = "[^a-zA-Z.]"
    doc = re.sub(regex, " ", doc)

    # Convert to lowercase
    doc = doc.lower()
        
    # Tokenization
    tokens = nltk.word_tokenize(doc)

    # Stop word removal
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize
    stemmer = PorterStemmer()

    clean_tokens_stem = [stemmer.stem(tokens) for tokens in filtered_tokens]
    
    # Join and return
    return " ".join(clean_tokens_stem)

In [16]:
df['clean_text2'] = df['text'].apply(lambda x : clean1(x))

df.head()

,label,text,char_token_count,word_token_count,clean_text,word_token_count_clean,clean_text2
0,MajorClaim,we should attach more importance to cooperatio...,72,10,attach importance cooperation primary education,5,attach import cooper primari educ
1,Premise,Take Olympic games which is a form of competit...,297,58,take olympic game form competition instance ha...,25,take olymp game form competit instanc hard ima...
2,Premise,The high technology and new ideas applied into...,154,26,high technology new idea applied practice may ...,15,high technolog new idea appli practic may lead...
3,Premise,pollutions are not just caused by the burning ...,137,20,pollution caused burning oil chemical pollutan...,13,pollut caus burn oil chemic pollut extra light...
4,Premise,the improvements of work efficiency also attri...,127,19,improvement work efficiency also attribute spe...,13,improv work effici also attribut speed work pa...


In [17]:
df["clean_text2"][0]

'attach import cooper primari educ'

In [18]:
df['clean_text2'].apply(lambda x: len(word_tokenize(x)))[0]

5

### Apply Vectorization using Bag of Word Technique(with default parameters) on the raw text column.

In [19]:
# import feature extraction methods from sklearn
from sklearn.feature_extraction.text import CountVectorizer

# instantiate a vectoriezer
bow_vect = CountVectorizer()

# use it to extract features from training data
text_dtm = bow_vect.fit_transform(df['clean_text'])


print(f"Shape of text_dtm (# of docs, # of unique vocabulary): {text_dtm.shape}")
print(f"Vocab: {bow_vect.get_feature_names_out()}")

Shape of text_dtm (# of docs, # of unique vocabulary): (6089, 5973)
Vocab: ['aaa' 'abandon' 'abbreviated' ... 'zone' 'zoo' 'zookeepers']


In [20]:
# import feature extraction methods from sklearn
from sklearn.feature_extraction.text import CountVectorizer

# instantiate a vectoriezer
bow_vect = CountVectorizer(ngram_range=(1, 1))

# use it to extract features from training data
text_dtm = bow_vect.fit_transform(df['clean_text'])


print(f"Shape of text_dtm (# of docs, # of unique vocabulary): {text_dtm.shape}")
print(f"Vocab: {bow_vect.get_feature_names_out()}")

Shape of text_dtm (# of docs, # of unique vocabulary): (6089, 5973)
Vocab: ['aaa' 'abandon' 'abbreviated' ... 'zone' 'zoo' 'zookeepers']


In [21]:
from sys import getsizeof

print(f"Size occupied by X_train_transformed in Memory: {getsizeof(text_dtm)} Bytes")

Size occupied by X_train_transformed in Memory: 48 Bytes


### Apply Vectorization using Bag of Word Vectorization(with following parameters) on the raw text column. 
1. token_pattern=None
2. tokenizer=callable 
 - use nltk word tokenizer
3. ngram_range=(1, **n)
4. lowercase=false
5. preprocessor=callable 
 - A valid token should only contain alphanumeric and "."
 - Convert to lower
 - word tokenize
 - stop word removal
 - lemmatization
6. stop_words=None

In [22]:
def tokenizer(doc):
    return nltk.word_tokenize(doc)

In [23]:
# import feature extraction methods from sklearn
from sklearn.feature_extraction.text import CountVectorizer

# instantiate a vectoriezer
bow_vect = CountVectorizer(token_pattern=None,
                           tokenizer=tokenizer, 
                           ngram_range=(1, 1), 
                           lowercase=False, 
                           preprocessor=clean, 
                           stop_words=None)

# use it to extract features from training data
text_dtms = bow_vect.fit_transform(df['text'])

print()
print(f"Shape of text_dtm (# of docs, # of unique vocabulary): {text_dtms.shape}")


Shape of text_dtm (# of docs, # of unique vocabulary): (6089, 5982)


In [24]:
# import feature extraction methods from sklearn
from sklearn.feature_extraction.text import CountVectorizer

# instantiate a vectoriezer
bow_vect = CountVectorizer(token_pattern=None,
                           tokenizer=tokenizer, 
                           ngram_range=(1, 2), 
                           lowercase=False, 
                           preprocessor=clean, 
                           stop_words=None)

# use it to extract features from training data
text_dtms = bow_vect.fit_transform(df['text'])

print()
print(f"Shape of text_dtm (# of docs, # of unique vocabulary): {text_dtms.shape}")


Shape of text_dtm (# of docs, # of unique vocabulary): (6089, 42023)


In [25]:
# import feature extraction methods from sklearn
from sklearn.feature_extraction.text import CountVectorizer

# instantiate a vectoriezer
bow_vect = CountVectorizer(token_pattern=None,
                           tokenizer=tokenizer, 
                           ngram_range=(1, 3), 
                           lowercase=False, 
                           preprocessor=clean, 
                           stop_words=None)

# use it to extract features from training data
text_dtms = bow_vect.fit_transform(df['text'])

print()
print(f"Shape of text_dtm (# of docs, # of unique vocabulary): {text_dtms.shape}")


Shape of text_dtm (# of docs, # of unique vocabulary): (6089, 79862)
